In [86]:
# PATH_NAME = 'C:/Users/gkstk/OneDrive/Desktop/SangMin/Github/AI/data/'
# PATH_NAME2 = 'C:/Users/gkstk/OneDrive/Desktop/SangMin/Github/AI/'


In [87]:
PATH_NAME = 'C:/Users/woobi/Documents/habit-AI/data/'
PATH_NAME2 = 'C:/Users/woobi/Documents/habit-AI/'

In [88]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import pickle
import math
from collections import Counter

In [96]:
model = KeyedVectors.load(PATH_NAME + "한국어_음식모델_한상민.kv", mmap='r')

with open(PATH_NAME + 'wweia_synonym_cats.pickle', 'rb') as handle:
  wweia_synonym_cats = pickle.load(handle)
  
wweia_food_categories = pd.read_csv(PATH_NAME + 'wweia_food_categories_addtl.csv')
wweia_data = pd.read_csv(PATH_NAME + 'wweia_data.csv')
wweia_embeddings = pd.read_csv(PATH_NAME + 'word_embeddings.csv', delimiter = ",")

In [90]:
stop_words = ['가', '걍', '것', '고', '과', '는', '도', '들', '등', '때', '로', '를', '뿐', '수', '아니', '않', '없', '에', '에게', '와', '으로', '은', '의', '이', '이다', '있', '자', '잘', '좀', '하다', '한', '조각', '개', '것', '대', '소' ,'단계', '등급', '포함', '미니', '개입']

In [91]:
def reduce_with_food_words(rough_phrase):
  korean_string = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", " ", rough_phrase)
  okt = Okt()
  token = okt.morphs(korean_string, stem=True)
  
  return token

In [92]:
def process_food_log(curr_log, wweia_synonym_cats):
  curr_log['predicted_categories_number'] = 0
  curr_log['predicted_categories_words'] = ""
  curr_log['max_cosim_score'] = 0
  curr_log['most_sim_food'] = ""
  curr_log['reciprocal_rank'] = 0.0
  curr_log['sym_reciprocal_rank'] = 0.0

  for i in range(curr_log.shape[0]):
    food_name = curr_log.loc[i, 'Food Name']
    pre_embedding = reduce_with_food_words(food_name)

    word_embed = np.zeros(shape = (1, len(model["불고기"])))
    if len(pre_embedding) > 0:
      
      num_words = 0
      for word in pre_embedding:
        word = word.lower()
        
        if word in model:
          num_words += 1
          word_embed += model[word]

      if num_words != 0:
        word_embed /= num_words
    
    # print("음식 이름")
    # print(food_name)
    # print("프리임베딩")
    # print(pre_embedding)
    # print(word_embed)
    # print('\n')
    
    similarities = cosine_similarity(word_embed, wweia_embeddings)
    to_keep = np.sort(similarities, axis=1)
    to_keep_args = np.argsort(similarities, axis=1)
    indices = np.flip(to_keep_args, axis = 1)
    
    sym_rank = 1000000
    for index in range(indices.shape[1]):
      true_cat = curr_log.loc[i, 'wweia_food_category_code']
      if math.isnan(true_cat): continue
      if wweia_data.loc[indices[0,index], 'NO'] == true_cat:
        rank = index
        rr = 1 / (rank+1)
        if sym_rank > rank:
          sym_rank = rank
          sym_rr = 1 / (rank+1)
        break
      else:
        
        if wweia_data.loc[indices[0,index], 'NO'] in wweia_synonym_cats[true_cat]:
          if sym_rank > index:
            sym_rank = index
            sym_rr = 1 / (index+1)
    most_sim_food_index = indices[0,0]

    most_sim_food_row = wweia_data.iloc[most_sim_food_index, :]
    highest_cat_num = most_sim_food_row['NO']
    highest_cat_words = wweia_food_categories.loc[wweia_food_categories['NO'] == highest_cat_num, '식품명']
    curr_log.loc[i, 'predicted_categories_number'] = highest_cat_num
    curr_log.loc[i, 'predicted_categories_words'] = highest_cat_words.to_list()[0]
    curr_log.loc[i, 'max_cosim_score'] = np.array2string(to_keep[0,-5:])
    curr_log.loc[i, 'most_sim_food'] = most_sim_food_row['식품명']
    curr_log.loc[i, 'reciprocal_rank'] = rr
    curr_log.loc[i, 'sym_reciprocal_rank'] = sym_rr
    
  return curr_log

In [93]:
# Main method
def food_recommandation(input_food_list) :
  df = pd.read_csv(PATH_NAME+ "wweia_food_categories_addtl.csv")
  
  numbers = [item[0] for item in input_food_list]
  input_data = df[df['NO'].isin(numbers)]
  print(input_data)
  print("--------------------------------------------------")
  
  input_list= ["wweia_food_category_code", "Food Name", "wweia_food_category_description"]
  curr_log = pd.DataFrame(input_food_list, columns=input_list)
  
  curr_log = process_food_log(curr_log, wweia_synonym_cats)

  print("true 출력 ")
  first_list = set(curr_log.loc[:,'wweia_food_category_code'].tolist())
  first_select_data = df[df['NO'].isin(first_list)]
  print(first_list)
  
  second_list = set(curr_log.loc[:,'predicted_categories_number'].tolist())
  second_select_data = df[df['NO'].isin(second_list)]
  print("pred 출력 ")
  print(second_list)
  
  print("--------------------------------------------------")
  print(first_select_data)
  print("--------------------------------------------------")
  print(second_select_data)

In [94]:
import pandas as pd
import random

# CSV 파일 읽기
df = pd.read_csv(PATH_NAME+ "wweia_food_categories_addtl.csv")

# 100개의 랜덤한 번호 선택
random_numbers = random.sample(range(1, 7684), 100)

# 선택된 번호에 해당하는 데이터 가져오기
selected_data = df[df['NO'].isin(random_numbers)]


# 필요한 열만 선택하여 리스트로 변환
food_list = selected_data[['NO', '식품명', '식품대분류']].values.tolist()

# 결과 출력
print(food_list)


[[39, '프리백', '과자류'], [152, '애니멀 쇼콜라', '과자류'], [177, '버터 가득 라이스 쿠키', '과자류'], [259, '조기구이', '구이류'], [499, '흑미도우', '기타'], [630, '땅콩, 삶은것, 소금첨가', '농축산물'], [689, '피스타치오, 볶은것', '농축산물'], [697, '호두, 볶은것', '농축산물'], [731, '멥쌀, 백미, 보람찬, 생것', '농축산물'], [1173, '롱안, 생것', '농축산물'], [1226, '보리밥 열매, 말린것', '농축산물'], [1312, '잭프루트, 통조림', '농축산물'], [1317, '체리, 통조림, 고형물', '농축산물'], [1320, '크랜베리, 말린것, 가당', '농축산물'], [1417, '달걀, 부침(달걀프라이)', '농축산물'], [1472, '조청, 오가피', '농축산물'], [1526, '동부, 수입산, 삶은것', '농축산물'], [1534, '완두, 삶은것', '농축산물'], [1593, '버들송이버섯, 갓, 말린것', '농축산물'], [1600, '상황버섯, 추출', '농축산물'], [1617, '영지버섯, 추출', '농축산물'], [1694, '까나리, 삶아서 말린것', '농축산물'], [1703, '꽁치, 통조림', '농축산물'], [1715, '노래미, 줄노래미, 생것', '농축산물'], [1744, '대구, 암컷, 생것', '농축산물'], [1802, '드렁허리, 생것', '농축산물'], [1922, '상어, 별상어, 생것', '농축산물'], [2004, '잉어, 생것', '농축산물'], [2445, '닭고기, 다리(아랫다리), 생것', '농축산물'], [2524, '돼지 부산물, 심장(염통), 생것', '농축산물'], [2612, '소고기, 한우(1++등급), 양지(업진살), 생것', '농축산물'], [2838, '토끼고기, 살코기, 생것', '농축산물'], [2860, '탄산 음료, 파인애플소다', '농축산물'], [2924

In [98]:
two_dimensional_array = [
    [1, "메밀전병", "곡류 및 서류"],
    [2, "약식", "곡류 및 서류"],
    [3, "무지개떡", "곡류 및 서류"],
    [45, "초코 마카롱", "과자류"],
    [106, "레몬머랭", "과자류"],
    [105, "코코넛머랭", "과자류"],
    [335, "홍합미역국", "국 및 탕류"]
]
food_recommandation(two_dimensional_array)

      NO     식품명    식품대분류   식품상세분류
0      1    메밀전병  곡류 및 서류  곡류 및 서류
1      2   수수부꾸미  곡류 및 서류  곡류 및 서류
2      3      약식  곡류 및 서류  곡류 및 서류
44    45  초코 마카롱      과자류   기타 과자류
104  105   코코넛머랭      과자류   기타 과자류
105  106    레몬머랭      과자류   기타 과자류
334  335   홍합미역국   국 및 탕류   어패류국.탕
--------------------------------------------------


C:\Users\woobi\AppData\Local\Temp\ipykernel_21572\3713454628.py:63: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.99717492 0.9972011  0.99731398 0.99734617 0.99830991]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  curr_log.loc[i, 'max_cosim_score'] = np.array2string(to_keep[0,-5:])


true 출력 
{1, 2, 3, 105, 106, 45, 335}
pred 출력 
{9, 237, 816, 2288, 55, 701, 2015}
--------------------------------------------------
      NO     식품명    식품대분류   식품상세분류
0      1    메밀전병  곡류 및 서류  곡류 및 서류
1      2   수수부꾸미  곡류 및 서류  곡류 및 서류
2      3      약식  곡류 및 서류  곡류 및 서류
44    45  초코 마카롱      과자류   기타 과자류
104  105   코코넛머랭      과자류   기타 과자류
105  106    레몬머랭      과자류   기타 과자류
334  335   홍합미역국   국 및 탕류   어패류국.탕
--------------------------------------------------
        NO                      식품명    식품대분류        식품상세분류
8        9                    송편(깨)  곡류 및 서류            떡류
54      55                  초코롱 산딸기      과자류        기타 과자류
236    237  3mm 황금비율로 바삭하고 고소한 전병세트      과자류           한과류
700    701          귀리, 쌀귀리, 도정, 생것     농축산물     곡류 및 그 제품
815    816          멥쌀, 현미, 해담쌀, 생것     농축산물     곡류 및 그 제품
2014  2015           장어, 뱀장어, 간, 생것     농축산물  어패류 및 기타 수산물
2287  2288               새우, 젓갈, 추젓     농축산물  어패류 및 기타 수산물
